In [1]:
!git clone https://github.com/chickowar/6-sem-CV-dz-4

Cloning into '6-sem-CV-dz-4'...
remote: Enumerating objects: 13, done.
remote: Counting objects: 100% (13/13), done.
remote: Compressing objects: 100% (11/11), done.
remote: Total 13 (delta 1), reused 12 (delta 0), pack-reused 0 (from 0)
Receiving objects: 100% (13/13), done.
Resolving deltas: 100% (1/1), done.


In [2]:
%cd 6-sem-CV-dz-4/

/content/6-sem-CV-dz-4


In [3]:
import sys
sys.path.append('./src') # чтобы он видел модуль src

In [4]:
from src.datasets import get_mnist, get_svhn
from src.models import SimpleCNN
from src.train_eval import train, evaluate

import torch
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cuda')

# Загрузка датасетов и модельки

In [10]:
# datasets.py
import torchvision.transforms as transforms
from torchvision.datasets import MNIST, SVHN
from torch.utils.data import DataLoader

def get_mnist(batch_size=128, train=True):
    transform = transforms.Compose([
        transforms.Resize((32, 32)),  # SVHN имеет размер 32x32
        transforms.ToTensor(),
        transforms.Normalize((0.5,), (0.5,))
    ])
    dataset = MNIST(root='./data', train=train, transform=transform, download=True)
    return DataLoader(dataset, batch_size=batch_size, shuffle=train)

def get_svhn(batch_size=128, split='test'):
    transform = transforms.Compose([
        transforms.Resize((32, 32)),
        transforms.Grayscale(),  # приводим к 1-канальному виду
        transforms.ToTensor(),
        transforms.Normalize((0.5,), (0.5,))
    ])
    dataset = SVHN(root='./data', split=split, transform=transform, download=True)
    return DataLoader(dataset, batch_size=batch_size, shuffle=False)

In [42]:
# models.py
import torch.nn as nn
import torch.nn.functional as F

class SimpleCNN(nn.Module):
    def __init__(self, num_classes=10):
        super().__init__()
        self.conv1 = nn.Conv2d(1, 32, 5)
        self.pool = nn.MaxPool2d(2)
        self.conv2 = nn.Conv2d(32, 64, 5)
        self.fc1 = nn.Linear(64 * 5 * 5, 256)
        self.fc2 = nn.Linear(256, num_classes)

    def forward(self, x):
        # print(f"fwd: {x.shape}")
        x = self.pool(F.relu(self.conv1(x)))  # [B, 32, 14, 14]
        # print(f"fwd: {x.shape}")
        x = self.pool(F.relu(self.conv2(x)))  # [B, 64, 5, 5]
        # print(f"fwd: {x.shape}")
        x = x.view(-1, 64 * 5 * 5)
        # print(f"fwd: {x.shape}")
        x = F.relu(self.fc1(x))
        # print(f"fwd: {x.shape}")
        return self.fc2(x)

In [43]:
# train_eval.py
import torch
import torch.nn as nn
import torch.optim as optim
from tqdm import tqdm

def train(model, dataloader, device, epochs=5, lr=1e-3):
    model.to(device)
    optimizer = optim.Adam(model.parameters(), lr=lr)
    criterion = nn.CrossEntropyLoss()

    model.train()
    for epoch in range(epochs):
        running_loss = 0.0
        for images, labels in tqdm(dataloader, desc=f"Epoch {epoch+1}/{epochs}"):
            images, labels = images.to(device), labels.to(device)
            # print(images.shape, labels.shape)

            optimizer.zero_grad()
            pred = model(images)
            # print(pred.shape)
            loss = criterion(pred, labels)
            loss.backward()
            optimizer.step()
            running_loss += loss.item()
        print(f"Epoch {epoch+1} - Loss: {running_loss / len(dataloader):.4f}")

def evaluate(model, dataloader, device):
    model.to(device)
    model.eval()
    correct, total = 0, 0
    with torch.no_grad():
        for images, labels in dataloader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            _, preds = outputs.max(1)
            correct += (preds == labels).sum().item()
            total += labels.size(0)
    acc = 100 * correct / total
    print(f"Accuracy: {acc:.2f}%")
    return acc


In [44]:
mnist_loader = get_mnist(train=True)
mnist_test = get_mnist(train=False)
svhn_test = get_svhn()
model = SimpleCNN()

Обучение на MNIST (и проверка на MNIST и SVHN)

In [45]:
train(model, mnist_loader, device, epochs=10)

Epoch 1/10: 100%|██████████| 469/469 [00:17<00:00, 27.54it/s]


Epoch 1 - Loss: 0.1650


Epoch 2/10: 100%|██████████| 469/469 [00:18<00:00, 25.74it/s]


Epoch 2 - Loss: 0.0442


Epoch 3/10: 100%|██████████| 469/469 [00:16<00:00, 27.79it/s]


...

Epoch 8 - Loss: 0.0091


Epoch 9/10: 100%|██████████| 469/469 [00:17<00:00, 26.21it/s]


Epoch 9 - Loss: 0.0098


Epoch 10/10: 100%|██████████| 469/469 [00:16<00:00, 27.96it/s]

Epoch 10 - Loss: 0.0063


In [46]:
print("Evaluation on MNIST (same domain):")
evaluate(model, mnist_test, device)

Evaluation on MNIST (same domain):
Accuracy: 99.26%


99.26

In [47]:
print("Evaluation on SVHN (cross-domain):")
evaluate(model, svhn_test, device)

Evaluation on SVHN (cross-domain):
Accuracy: 37.31%


37.30792870313461